In [1]:
# 엘라스틱서치의 데몬 인스턴스 만들기
import os
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import json
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['/home/pervinco/elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                #   preexec_fn=lambda: os.setuid(1) 
                #  # as daemon
                 )

import time 

time.sleep(30)

In [2]:
# Elasticsearch 접속 정보
username = 'elastic'
password = 'DrZEurHL5sxLmS5PwGXs'

# Elasticsearch 클라이언트 생성 (HTTP로 접속)
es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/home/pervinco/elasticsearch-8.8.0/config/certs/http_ca.crt")

# Elasticsearch 서버 정보 확인
resp = dict(es.info())
print(resp)


{'name': 'pervinco-B650-PG-Lightning', 'cluster_name': 'elasticsearch', 'cluster_uuid': 't-JIPY0BQd6DTDiHbmmacA', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


### 데이터 전처리

In [3]:
# 위키미디어로부터 kowiki 데이터를 다운로드 받음
# !wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
# 위키데이터의 노이즈를 제거하고 json 형태로 반환하는 코드를 참조
# !git clone https://github.com/attardi/wikiextractor.git
# 다운로드 받은 샘플 위키 데이터를 전처리하여 검색의 입력으로 사용
# 결과는 elastic 폴더에 'extract_result/AA,AB,AC.../wiki_00..99'라는 새로운 폴더에 저장된다.(용량이 비슷하게 나눠서 저장됨)
# 변환결과 wiki_00 파일의 내용 샘플  {"id": "5", "revid": "641228", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "\uc9c0\...\ud130", "text": "\uc81c\...\ub2e4."}
# !python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles1.xml-p1p82407.bz2 --json -o extract_result

In [4]:
import json
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("hunkim/sentence-transformer-klue")

def get_embedding(sentences):
    # 입력 문장을 인코딩하여 임베딩을 얻음
    return model.encode(sentences)

wiki_dump_json_file = '/home/pervinco/Datasets/extract_result/AA/wiki_00'
# 'wiki_dump_json_file'에 있는 JSON 파일 읽어들여 index_docs에 저장

index_docs = []
for line in open(wiki_dump_json_file, encoding="utf-8"):
    # JSON 데이터를 읽어들여 파이썬 딕셔너리로 변환
    json_data = json.loads(line)

    # 'text'에 대한 임베딩을 계산하여 'embeddings' 필드에 추가
    json_data['embeddings_text'] = get_embedding(json_data['text']).tolist()
    json_data['embeddings_title'] = get_embedding(json_data['title']).tolist()

    # 색인할 문서 목록에 추가
    index_docs.append(json_data)

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## 2. 색인 및 검색

In [17]:
# Elasticsearch 색인/검색을 위한 공통 함수 정의

from elasticsearch import Elasticsearch, helpers
import json
import pprint as pp

def create_es_index(index, body):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, body=body)

def delete_es_index(index):
    # 지정된 인덱스 삭제
    es.indices.delete(index=index)

def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    # Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
    return helpers.bulk(es, actions)

def sparse_retrieve(condition, index):
    # 지정된 인덱스에서 생성된 쿼리를 사용하여 검색 수행 (역색인을 이용한 일반 검색)
    return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")

def dense_retrieve(condition, index):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([condition["query"]])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": condition["field"],
        "query_vector": query_embedding.tolist(),
        "k": condition["size"],
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index=index, knn=knn)


In [18]:
# 색인을 위한 mapping 설정
setting = {
    "settings": {
        "analysis": {
            "analyzer": {
                "nori": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "decompound_mode": "mixed",
                    "filter": ["nori_posfilter"]
                }
            },
            "filter": {
                "nori_posfilter": {
                    "type": "nori_part_of_speech",
                    # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                    "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            # 비교 테스트를 위해 meta field를 같이 색인
            "text": {"type": "text", "analyzer": "nori"},
            "title": {"type": "text", "analyzer": "nori"},
            "embeddings_title": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "l2_norm"
            },
            "embeddings_text": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "l2_norm"
            }
        }
    }
}

In [19]:
# 'setting'으로 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", setting)

# 'test' 인덱스에 대량 색인화 수행
# 각 문서는 'embbedings' 라는 필드를 가짐
ret = bulk_add("test", index_docs)

# 결과 출력
print(ret)

/tmp/ipykernel_573414/3329331361.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index, body=body)


(61, [])


In [20]:
# 역색인을 사용하는 검색 예제
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "text": {
                    "query": "문재인의 친구"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

/tmp/ipykernel_573414/3329331361.py:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [21]:
# 역색인을 사용하는 검색 예제
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "text": {
                    "query": "문재인의 친구",
                    "operator": "AND"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 4.439573 source:: {'id': '85', 'revid': '805745', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원을 지냈고, 김대중 정부에서 제6대 해양수산부 장관을 지냈다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시행한 것, 대

/tmp/ipykernel_573414/3329331361.py:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [22]:
# Vector 유사도 사용한 검색 예제
condition_retrieve = {
    "field": "embeddings_text",
    "query": "문재인의 친구",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.0061784643 source:: {'id': '85', 'revid': '805745', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원을 지냈고, 김대중 정부에서 제6대 해양수산부 장관을 지냈다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시행한 

In [23]:
# 역색인을 사용하는 검색 예제 - title field 사용
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "title": {
                    "query": "문재인의 친구"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

/tmp/ipykernel_573414/3329331361.py:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [24]:
# Vector 유사도 사용한 검색 예제 - title field 사용
condition_retrieve = {
    "query": "문재인의 친구",
    "field": "embeddings_title",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.0073537673 source:: {'id': '85', 'revid': '805745', 'url': 'https://ko.wikipedia.org/wiki?curid=85', 'title': '노무현', 'text': '노무현(盧武鉉, 1946년 9월 1일~2009년 5월 23일)은 대한민국의 제16대 대통령이다. 판사로 재직 후 부산에서 변호사로 활동하다가 제13·15대 국회의원을 지냈고, 김대중 정부에서 제6대 해양수산부 장관을 지냈다.\n본관은 광주(光州)이며 경상남도 김해 출생이다. 부산상업고등학교를 졸업하고 막노동에 뛰어들었다가 독학으로 1975년 3월 30세에 제17회 사법시험에 합격하였다. 대전지방법원 판사로 1년을 재직하다가 그만두고 부산에서 변호사 사무실을 개업하여 여러 인권 사건을 변호하였다. 통일민주당 총재 김영삼의 공천을 받아 제13대 총선에 출마하여 부산 동구에서 당선되며 5공비리특별위원으로 활동했다. 1990년 3당 합당에 반대하면서 김영삼과 결별한다. 김대중 정부에서 해양수산부 장관을 지냈고 국민경선제에서 새천년민주당 소속으로 제16대 대선에서 대통령으로 당선되었으나 2003년 말에 새천년민주당을 탈당하고 2004년 초 새천년민주당을 탈당한 개혁 세력들이 주축이 되어 창당한 열린우리당에 입당하였다.\n2004년 무렵 공직선거 및 선거부정방지법이 정한 중립의무 및 헌법 위반을 시유로 야당에 국회로부터 대한민국 헌정 사상 최초로 대통령직 재임 중 탄핵 소추를 당해 대통령 직무가 정지되었다. 하지만 이후 탄핵을 주도했던 새천년민주당과 한나라당, 자유민주연합은 여론의 역풍에 휩싸여 제17대 총선에서 참패하였고 얼마 후 헌법재판소에서 소추안을 기각하며 노무현은 다시 대통령 직무에 복귀하였다.\n주요 업적으로는 권력층에 만연해 있던 권위주의와 정경유착을 타파하고 기존 정권이 하지 못했던 각종 재벌 개혁을 시행한 것이 꼽힌다. 상속증여세의 포괄주의를 도입해 대기업 총수의 탈세 여지를 좁힌 것, 증권 관련 집단소송제를 시행한 

In [25]:
# 역색인을 사용하는 검색 예제
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "title": {
                    "query": "우리나라 열 여섯번째 대통령이 누구야?"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 3.3614042 source:: {'id': '20', 'revid': '514184', 'url': 'https://ko.wikipedia.org/wiki?curid=20', 'title': '나라 목록', 'text': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n기준.\n위 기준에 논거하여 이 목록은 다음 208개 국가를 포함하고 있다.\n미승인 국가.\n이 목록은 주권을 주장하고 점유한 영토를 실제로 관리하고 있으나, 많은 국가와 외교관계를 맺지 못한 나라를 설명하고 있다. 마이크로네이션는 이 목록에 포함하지 않는다.', 'embeddings_text': [-0.1089089959859848, -0.28003233671188354, -0.03679170459508896, -0.2587352991104126, -0.23113475739955902, -0.48579615354537964, -0.0031198770739138126, -0.27645817399024963, -0.1901564598083496, -0.3604002296924591, 0.19388249516487122, 0.30622684955596924, 0.13646596670150757, -0.08949083834886551, -0.48911139369010925, 0.24047446250915527, -

/tmp/ipykernel_573414/3329331361.py:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [26]:
# Vector 유사도 사용한 검색 예제
condition_retrieve = {
    "query": "우리나라 열 여섯번째 대통령이 누구야?",
    "field": "embeddings_title",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.006525171 source:: {'id': '34', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=34', 'title': '대한민국 제16대 대통령 선거', 'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 해양수산부 장관 출신 노무현 새천년민주당 후보의 양강 구도로 진행되었다.\n대선 재수생인 이회창 후보는 경험이나 세력 면에서 노무현 후보보다 대권 고지에 좀 더 유리할 것으로 점쳐졌으나, 이전 대선부터 불거진 이회창 후보의 두 아들의 병역기피 논란, 노사모를 비롯한 네티즌들의 열성적인 노무현 지지, 정몽준 후보와의 단일화 성공 등에 힘입어 노무현 후보가 당선되었다.\n선거 정보.\n선거권.\n만 20세 이상의 대한민국 국민은 선거권이 있었다. 즉, 1982년 12월 19일 이전에 태어난 사람은 투표를 할 자격이 있었다.\n피선거권.\n만 40세 이상의 대한민국 국민은 피선거권을 가졌다. 즉, 1962년 12월 19일 이전에 태어난 사람은 후보자가 될 자격이 있었다.\n후보.\n새천년민주당.\n새천년민주당은 3월 9일부터 4월 27일까지 한국 정당 역사상 최초로 국민 참여 경선을 실시하고 과반 득표자인 노무현 전, 판사, 변호사, 해양수산부 장관을 대통령 후보로 선출하였다.\n한나라당.\n한나라당은 4월 13일부터 5월 9일까지 국민 참여 경선을 실시하고 최다 득표자인 이회창 전 당 총재를 대통령 후보로 선출하였다.\n민주노동당.\n민주노동당은 9월 8일 당원들에 의한 단일 후보 찬반 투표를 통해 권영길 당 대표를 대통령 후보로 선출하였다.\n국민통합21.

In [27]:
# 역색인을 사용하는 검색 예제 - 오탈자
condition_retrieve = {
    "query_body": {
        "query": {
            "match": {
                "title": {
                    "query": "우리나라 열 여섯번째 대통려이 누구야?"
                }
            }
        }
    },
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = sparse_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 3.3614042 source:: {'id': '20', 'revid': '514184', 'url': 'https://ko.wikipedia.org/wiki?curid=20', 'title': '나라 목록', 'text': '이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.\n특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.\n기준.\n위 기준에 논거하여 이 목록은 다음 208개 국가를 포함하고 있다.\n미승인 국가.\n이 목록은 주권을 주장하고 점유한 영토를 실제로 관리하고 있으나, 많은 국가와 외교관계를 맺지 못한 나라를 설명하고 있다. 마이크로네이션는 이 목록에 포함하지 않는다.', 'embeddings_text': [-0.1089089959859848, -0.28003233671188354, -0.03679170459508896, -0.2587352991104126, -0.23113475739955902, -0.48579615354537964, -0.0031198770739138126, -0.27645817399024963, -0.1901564598083496, -0.3604002296924591, 0.19388249516487122, 0.30622684955596924, 0.13646596670150757, -0.08949083834886551, -0.48911139369010925, 0.24047446250915527, -

/tmp/ipykernel_573414/3329331361.py:33: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index=index, body=condition["query_body"], size=condition["size"], sort="_score")


In [28]:
# Vector 유사도 사용한 검색 예제 - 오탈자
condition_retrieve = {
    "query": "우리나라 16대 대통려이 누구야?",
    "field": "embeddings_title",
    "size": 5  # Specify the number of documents to retrieve
}
search_result_retrieve = dense_retrieve(condition_retrieve, "test")

# 결과 출력
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])

score: 0.0113131525 source:: {'id': '34', 'revid': '368112', 'url': 'https://ko.wikipedia.org/wiki?curid=34', 'title': '대한민국 제16대 대통령 선거', 'text': '대한민국 제16대 대통령 선거는 2002년 12월 19일 목요일 치뤄진 대통령 선거로, 21세기에 처음으로 치뤄진 대한민국 대통령 선거이다. 제15대 김대중 대통령의 차기 대통령을 뽑기 위한 선거이다.\n16대 대선은 지난 15대 대선에서 간발의 차로 낙선하고 재도전한 이회창 한나라당 후보와 사상 최초의 국민 참여 경선을 통해 여당의 대통령 후보가 된 해양수산부 장관 출신 노무현 새천년민주당 후보의 양강 구도로 진행되었다.\n대선 재수생인 이회창 후보는 경험이나 세력 면에서 노무현 후보보다 대권 고지에 좀 더 유리할 것으로 점쳐졌으나, 이전 대선부터 불거진 이회창 후보의 두 아들의 병역기피 논란, 노사모를 비롯한 네티즌들의 열성적인 노무현 지지, 정몽준 후보와의 단일화 성공 등에 힘입어 노무현 후보가 당선되었다.\n선거 정보.\n선거권.\n만 20세 이상의 대한민국 국민은 선거권이 있었다. 즉, 1982년 12월 19일 이전에 태어난 사람은 투표를 할 자격이 있었다.\n피선거권.\n만 40세 이상의 대한민국 국민은 피선거권을 가졌다. 즉, 1962년 12월 19일 이전에 태어난 사람은 후보자가 될 자격이 있었다.\n후보.\n새천년민주당.\n새천년민주당은 3월 9일부터 4월 27일까지 한국 정당 역사상 최초로 국민 참여 경선을 실시하고 과반 득표자인 노무현 전, 판사, 변호사, 해양수산부 장관을 대통령 후보로 선출하였다.\n한나라당.\n한나라당은 4월 13일부터 5월 9일까지 국민 참여 경선을 실시하고 최다 득표자인 이회창 전 당 총재를 대통령 후보로 선출하였다.\n민주노동당.\n민주노동당은 9월 8일 당원들에 의한 단일 후보 찬반 투표를 통해 권영길 당 대표를 대통령 후보로 선출하였다.\n국민통합21

#Reference

## Required Package

sentence_transformers=2.2.2 <br>
elasticsearch==8.8.0
